In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC
import joblib

In [2]:
# Set Path Here
WORKING_DIRECTORY = r"F:\medical_images\alzeimer"
CLASSES = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']

In [3]:
# Data preparation
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_dataset = datagen.flow_from_directory(WORKING_DIRECTORY, target_size=(150, 150),
                                            batch_size=6400, shuffle=True)

Found 6400 images belonging to 4 classes.


In [4]:
# Separate data and labels
X, y = train_dataset.next()
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

In [5]:
# Print dataset info
print("Number of samples after splitting into Training, validation & test set\n")
print("Train     \t", sorted(Counter(np.argmax(y_train, axis=1)).items()))
print("Validation\t", sorted(Counter(np.argmax(y_val, axis=1)).items()))
print("Test      \t", sorted(Counter(np.argmax(y_test, axis=1)).items()))
print("Number of samples in each class:\t", sorted(Counter(np.argmax(y, axis=1)).items()))
print("Classes Names according to index:\t", train_dataset.class_indices)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

Number of samples after splitting into Training, validation & test set

Train     	 [(0, 623), (1, 47), (2, 2240), (3, 1570)]
Validation	 [(0, 224), (1, 14), (2, 773), (3, 525)]
Test      	 [(0, 49), (1, 3), (2, 187), (3, 145)]
Number of samples in each class:	 [(0, 896), (1, 64), (2, 3200), (3, 2240)]
Classes Names according to index:	 {'Mild_Demented': 0, 'Moderate_Demented': 1, 'Non_Demented': 2, 'Very_Mild_Demented': 3}
X_train shape: (4480, 150, 150, 3)
y_train shape: (4480, 4)
X_test shape: (384, 150, 150, 3)
y_test shape: (384, 4)


In [6]:
# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(len(CLASSES), activation='softmax')
])

In [7]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Train the CNN model
history = cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
140/140 [==============================] - 69s 478ms/step - loss: 1.0572 - accuracy: 0.5230 - val_loss: 0.8794 - val_accuracy: 0.5781
Epoch 2/10
140/140 [==============================] - 65s 461ms/step - loss: 0.8553 - accuracy: 0.6170 - val_loss: 0.8268 - val_accuracy: 0.6387
Epoch 3/10
140/140 [==============================] - 66s 471ms/step - loss: 0.6786 - accuracy: 0.6971 - val_loss: 0.5545 - val_accuracy: 0.7780
Epoch 4/10
140/140 [==============================] - 68s 484ms/step - loss: 0.4325 - accuracy: 0.8154 - val_loss: 0.3954 - val_accuracy: 0.8281
Epoch 5/10
140/140 [==============================] - 67s 475ms/step - loss: 0.2394 - accuracy: 0.9071 - val_loss: 0.2165 - val_accuracy: 0.9258
Epoch 6/10
140/140 [==============================] - 65s 464ms/step - loss: 0.1215 - accuracy: 0.9587 - val_loss: 0.1744 - val_accuracy: 0.9440
Epoch 7/10
140/140 [==============================] - 63s 451ms/step - loss: 0.0768 - accuracy: 0.9705 - val_loss: 0.3807 - val_ac

In [9]:
# Print training accuracy
print("Training Accuracy:")
print("Final Training Accuracy: {:.2f}%".format(history.history['accuracy'][-1] * 100))

# Print validation accuracy
print("Validation Accuracy:")
print("Final Validation Accuracy: {:.2f}%".format(history.history['val_accuracy'][-1] * 100))

Training Accuracy:
Final Training Accuracy: 99.75%
Validation Accuracy:
Final Validation Accuracy: 96.16%


In [10]:
# Save the CNN model
cnn_model.save("F:\medical_images\cnn-svm\cnn_model.h5")

In [14]:
from tensorflow.keras.models import Model

In [15]:
# Load the trained CNN model
cnn_model = tf.keras.models.load_model("F:\medical_images\cnn-svm\cnn_model.h5")

# Remove the last layer (softmax) to get feature vectors
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)

In [16]:
# Extract features for training and test data
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

12/12 [==============================] - 2s 107ms/step


In [17]:
# Flatten the features
train_features_flat = train_features.reshape((train_features.shape[0], -1))
test_features_flat = test_features.reshape((test_features.shape[0], -1))

In [18]:
# Train the SVM
svm = SVC(kernel='linear')
svm.fit(train_features_flat, np.argmax(y_train, axis=1))

SVC(kernel='linear')

In [19]:
# Save the SVM model
joblib.dump(svm, "F:\medical_images\cnn-svm\svm_model.pkl")

['F:\\medical_images\\cnn-svm\\svm_model.pkl']

In [20]:
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import joblib

# Load the trained CNN model
cnn_model = tf.keras.models.load_model("F:\medical_images\cnn-svm\cnn_model.h5")

# Load the feature extractor model
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)

# Load the SVM model
svm = joblib.load("F:\medical_images\cnn-svm\svm_model.pkl")

# Class names
CLASSES = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']

# Load and preprocess the sample image
img_path = r"F:\medical_images\alzeimer\Non_Demented\non_41.jpg"
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize pixel values to [0, 1]

# Extract features
features = feature_extractor.predict(img_array)
features_flat = features.flatten().reshape(1, -1)

# Predict with SVM
predicted_class_index = svm.predict(features_flat)[0]
decision_function = svm.decision_function(features_flat)
confidence = np.max(np.abs(decision_function))  # Measure of confidence

# Get the class name
predicted_class_label = CLASSES[predicted_class_index]

print("Predicted Class Index:", predicted_class_index)
print("Predicted Class:", predicted_class_label)
print("Confidence:", confidence)


1/1 [==============================] - 0s 125ms/step
Predicted Class Index: 2
Predicted Class: Non_Demented
Confidence: 3.280354309276325


In [22]:
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# Load the trained CNN model
cnn_model = tf.keras.models.load_model("F:/medical_images/cnn-svm/cnn_model.h5")

# Define the class names
class_names = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']

# Load and preprocess the sample image
img_path = r"F:\medical_images\alzeimer\Non_Demented\non_41.jpg" # Update this with your image path
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize pixel values to [0, 1]

# Predict the class probabilities
predictions = cnn_model.predict(img_array)

# Get the predicted class index and the confidence score
predicted_class_index = np.argmax(predictions[0])
confidence = predictions[0][predicted_class_index]

# Print the results
print(f"Predicted Class Index: {predicted_class_index}")
print(f"Predicted Class: {class_names[predicted_class_index]}")
print(f"Confidence: {confidence}")

1/1 [==============================] - 0s 78ms/step
Predicted Class Index: 2
Predicted Class: Non_Demented
Confidence: 0.9977602958679199
